#### This basic tutorial shows you how to do PWN control with STM32 Nucleo-L432KC. The datasheet of the TCS3425 serises can be found [here](https://cdn-shop.adafruit.com/datasheets/TCS34725.pdf). In this tutorial, we will show how to setup the a Timer on Nucleo-L432KC to output a PWM, and use the on-board LED to show the PWM.

## 1. Wiring

<img src="Figures\STM_pinout.png" alt="STM32 Nucleo-L432KC Pinout" width="800"/>

Connect PA_8 (D9) <--> PB_3 (D13), we will use PA_8 to output the PWM and use PB_3 to read it, which is internally connected to the on-board LED (Green).

## 2. CubeIDE Clock Configuration

We are going to use GPIO and the Timer `TIM1` (see [STM32 datasheet](https://www.st.com/resource/en/datasheet/stm32l432kb.pdf) page 38/156 Table 9) to generate a PWM with a desired frequency. To set the Timer frequency, we first need to set the HCLK frequency and determine the correct Prescaler to use for the Timer. Open the `.ioc` file in your project and click on the `Clock Configuration` tab to check the clock setting of the STM32.

<img src="Figures\Clock_setting.png" alt="Clock setting" width="1500"/>

The `APB2 timer clocks (MHz)` is the clock used for `TIM1` (see [STM32 datasheet](https://www.st.com/resource/en/datasheet/stm32l432kb.pdf) page 32/156 Figure 4)

<img src="Figures\Clock_tree.png" alt="Clock tree" width="500"/>

In my case, the `HCLK` is set to $32$ MHz, the `APB2 Prescaler` is set to $1$ and the `Multiplier` is set to $1$, so the `APB2 timer clocks (MHz)` is $32/1 \times 1=32$ MHz. Assume that we want the Timer to run at $10$ Hz, we can set the `APB2 timer clocks (MHz)` to $0.5$ MHz, then set a prescaler of $50000$ for the TIM1. (i.e., TIM1 frequency = $0.5e6/50000=10$ Hz.) If you prefer a different frequency, set the clock accordingly.

Change the value of the `APB2 timer clocks (MHz)` to $0.5$ then press Enter. You will notice the CubeIDE automatically updates other corresponding clocks.

<img src="Figures\PWM_clock_setting.png" alt="Clock setting" width="1500"/>

## 3. CubeIDE Pinout & Configuration

Go to the `Pinout & Configuration` tab, under `Timers`, select `TIM1`. Set the `Clock Source` to `Internal Clock`. Set the `Channel 1` to `PWM Generation CH1`. In the Configuration, set the `Prescaler` to $50000-1$. (The `Prescaler` value will be $n+1$ where $n$ is the number you enter.) Set `auto-reload preload` to `Enable`, which allows the Timer to reset the counter after it finishes the current round. This way, the Timer will run continuously without interruption, smoothly transitioning from the end of one period to the beginning of the next. In the `Pinout review` window, you should now have `PB3` as a `GPIO_Input` by default, and `PA8` labelled as `TIM1_CH1` for the output of `TIM1` from Channel 1. (There is a chance that your IDE picks a different pin for TIM1_CH1, in this case, change the corresponding hardware and software configurations.)

<img src="Figures\PWM_pin_setting.png" alt="Pin configuration" width="1500"/>

In order to debug, in the `System Core` dropdown list, click `SYS` and make sure the `Debug` mode is set to `Serial Wire`. (The figure below is not from this project, ignore the Pinout review.)

<img src="Figures\ultrasound_debug.png" alt="Debug configuration" width="1500"/>

After setting, save the file and generate code.

## 4. Coding

Now our Timer `TIM1` has been set to have a frequency of $10$ Hz. To control the PWM, we configurate two registers:
- AutoReload Register (`ARR`): 
    - defines the value at which the timer counter resets, thereby determining the timer's period and controlling the frequency of PWM signals generated by the timer.
- The Capture/Compare Register (`CCR`): 
    - is used to store values that control various aspects of timer operation, such as setting the compare value for PWM generation, capturing the timer's counter value at specific events, or setting the pulse width in PWM mode.

Let's assume we want to generate a PWM with a period of $1$ s and a duty cycle (DC) of $50$%, we can set `ARR` to $10$ and `CRR` to $5$. In this way, each PWM preiod is $10$ Timer preiod ($1/10*10=1$ s). And for the first $5$ Timer preiod, the Timer channel 1 outputs a `HIGH` (And the rest `LOW`), which achieves a DC of $50$%.

In this tutorial, let's program the STM32 to generate a PWM with a frequency of $1$ Hz and a DC of $50$%. Then after $3$ s, change the frequency to $2.5$ Hz. After another $3$ s, change the frequency back to $1$ Hz, and so on.

First, we need to start the PWM generator. Put the following code between `/* USER CODE BEGIN 2 */` and `/* USER CODE END 2 */`:

```c
  /* USER CODE BEGIN 2 */
  HAL_TIM_PWM_Start(&htim1, TIM_CHANNEL_1);
  /* USER CODE END 2 */
```

This specifies which Timer (`TIM1`) we want to generate PWM, and which channel pin (`TIM1_CH1`) we want to output the PWM.

Now copy the following code between `/* USER CODE BEGIN WHILE */` and `/* USER CODE END WHILE */`:

```c
  /* USER CODE BEGIN WHILE */
  while (1)
  {
	TIM1->ARR = 10;
	TIM1->CCR1 = 5;
	HAL_Delay(3000);
	TIM1->ARR = 4;
	TIM1->CCR1 = 2;
	HAL_Delay(3000);
    /* USER CODE END WHILE */

    /* USER CODE BEGIN 3 */
  }
```

Read this code, and try to understand each line.

## 5. Debugging

Now, click on `Debug` to debug the program. (Don't forget to click `resume` to run the program.) When the program is running, you will see the on-board LED blinking with a frequency of $1$ Hz for $3$ seconds, then $2.5$ Hz for $3$ seconds and so on. Feel free to play with the code.